In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import window, col
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, ShortType, DateType
from time import sleep

# Set up Spark session
sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("count_window")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")

# Create the spark session
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

####################
# Get data
## Create schema
dataSchema = StructType(
    [StructField("city", StringType(), True),
     StructField("datetime", DateType(), True)
])

## Read from a source - File: https://raw.githubusercontent.com/DeskThom/DE2024_assignment2/refs/heads/main/stream/data/outage_data_20241127_121735.json
sdf = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1) \
        .json("/home/jovyan/data/stream-data")

sdf.printSchema()

##################


sdf.groupBy(window(col("datetime"), "1 day"), "city").count() \
        .writeStream \
        .queryName("outages_per_window") \
        .format("memory") \
        .outputMode("complete") \
        .start()

for x in range(10):
    spark.sql("SELECT * FROM outages_per_window").show()
    sleep(10)
